### Import des bibliothèques



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import warnings

from arch import arch_model
    
from datetime import datetime, timedelta
#import datetime

# import tensorflow as tf

from numpy import array

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.stattools import arma_order_select_ic

from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

from statsmodels.tsa.holtwinters import SimpleExpSmoothing 
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from statsmodels.graphics.tsaplots import plot_pacf

import os
import csv

import math

from IPython.display import Markdown, display

def printmd(string):
    display(Markdown(string))

ModuleNotFoundError: No module named 'pandas'

### Classe Loader

Classe pour charger les valeurs mobilières.

In [ ]:
class Loader :

    # 1.ajouter vos valeurs mobilières ici
    ADA = "Cardano"
    AAVE = "Aave"
    BTC = "Bitcoin"
    COSMOS = "Cosmos"
    DOGECOIN = "Dogecoin"
    ETH = "Ethereum"
    LITECOIN = "Litecoin"
    MONERO = "Monero"
    POLKADOT = "Polkadot"
    USDCOIN = "USDCoin"

    MICROSOFT = "Microsoft"
    AMAZON = "Amazon"
    ATT = "AT&T"
    CAC40 = "cac40"
    TESLA = "Tesla"

    # 2. ajouter la variable dans la liste
    SECURITY = [ADA, AAVE, BTC, COSMOS, DOGECOIN, ETH, LITECOIN, MONERO, POLKADOT, USDCOIN,
                MICROSOFT, ATT, CAC40, AMAZON, TESLA]

    # 3. ajouter votre source ici (dossier `nouvelleSource` dans `data`)
    COINCODEX = "Coincodex"
    YAHOO = "finance.yahoo.com"
    KAGGLE = "Kaggle"
    NASDAQ = "Nasdaq"
    BINANCE = "Binance"

    # 4. ajouter votre source dans la liste
    SOURCES = [BINANCE, COINCODEX, YAHOO, KAGGLE, NASDAQ]

    # 5. ajouter votre nom de base de fichier pour la nouvelle source (la clé est la source)
    BASE_NAME = {
        BINANCE : "",
        COINCODEX : "coin_",
        YAHOO : "",
        KAGGLE : "coin_",
        NASDAQ : ""
    }

    # Granularity Day, Hour
    GRANULARIY = ['', 'h']

    def __init__(self, debug=False):
        self.data = {}
        self.debug = debug

        for source in Loader.SOURCES :
            self.data[source] = {}

    def loadFile(self, source, security, granularity='', dateStart=None, dateEnd=None):
        '''
        @param granulariy: default is day. Possible value is 'h'
        
        '''
        fileName = Loader.BASE_NAME[source]+security
        
        if(granularity != ''):
            fileName = fileName+'-'+granularity
        
        fileName = fileName+".csv"

        path = os.path.join("data", source, fileName)

        # print(path)
        
        if(os.path.isfile(path)):
            df = pd.read_csv(path,
                                header=0)
            
            df["Date"] = pd.to_datetime(df["Date"])
            df["Date"] = df["Date"].dt.date
            df= df.set_index("Date")

            df.sort_index(axis=0, inplace=True)
            
            # print(source, " - ", df.columns)
            
            if source == Loader.BINANCE:
                # print("---------------", df.columns)
                df = df[['high', 'low', 'open', 'Close', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']
                df['Market Cap'] = None
            elif source == Loader.COINCODEX:
                df = df[['High', 'Low', 'Open', 'Close', 'Volume', 'Market Cap']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']
            elif source == Loader.KAGGLE:
                df = df[['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume', 'Marketcap']
            elif source == Loader.NASDAQ:
                # ['Close/Last', 'Volume', 'Open', 'High', 'Low']
                df = df[['High', 'Low', 'Open', 'Close/Last', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']
            elif source == Loader.YAHOO:
                # ['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

                df = df[['High', 'Low', 'Open', 'Close', 'Volume']]
                df.columns = ['High', 'Low', 'Open', 'Close', 'Volume']

            if(dateStart and dateEnd):
                dateStart = datetime.strptime(dateStart, '%Y-%m-%d').date()
                dateEnd = datetime.strptime(dateEnd, '%Y-%m-%d').date()
                df = df.loc[dateStart:dateEnd]

            self.data[source][security] = df
            
            if self.debug:
                print(source, "[", security, "]"," charged")

        else:
            if self.debug:
                print(source, "[", security, "]"," not charged")


    def loadSource(self, source, dateStart=None, dateEnd=None, forceEqualDate=False):
        
        if self.debug:
            print("-----------------")
        
        for security in Loader.SECURITY:
                self.loadFile(source, security, dateStart=dateStart, dateEnd=dateEnd)

        # print("-----------------", ((dateStart and dateEnd) or forceEqualDate))
        
        
        if( (dateStart and dateEnd) or forceEqualDate):
            startDates = []        
            endDates =  []

            for security, df in self.data[source].items():
                startDates.append(df.index[0])
                endDates.append(df.index[-1])

            s = max(startDates)
            e = min(endDates)

            for security, df in self.data[source].items():
                self.data[source][security] = df.loc[s:e]
        
    def loadSources(self, dateStart=None, dateMax=None):
        for source in Loader.SOURCES:
            self.loadSource(source)

    def get(self, source, nameSecurity):
        return self.data[source][nameSecurity]



### Exemple d'utilisation

Pour charger une données précises, il faut indiquer :

- la source des données (un dossier par source dans le dossier `data`)
- le nom de la valeur mobilière
- en option, les dates sur lesquels charger la donnée



In [ ]:
loader = Loader()

loader.loadFile(Loader.COINCODEX, Loader.ADA, dateStart="2000-01-01", dateEnd="2024-01-01")

df = loader.get(Loader.COINCODEX, Loader.ADA)

print(df.dtypes)
print("df min date:", df.index[0], " max date:", df.index[-1])
df.head()




High         float64
Low          float64
Open         float64
Close        float64
Volume       float64
Marketcap    float64
dtype: object
df min date: 2017-10-01  max date: 2022-08-17


,High,Low,Open,Close,Volume,Marketcap
Date,,,,,,
2017-10-01,0.030088,0.019969,0.024969,0.025808,8.310030e+07,5.672757e+08
2017-10-02,0.027172,0.020575,0.025671,0.020827,2.971484e+07,6.786349e+08
2017-10-03,0.022962,0.020828,0.020828,0.021973,1.243013e+07,5.695444e+08
2017-10-04,0.022245,0.020839,0.022035,0.021512,7.055960e+06,5.548686e+08
2017-10-05,0.021576,0.018071,0.021286,0.018481,6.207887e+06,5.219246e+08


Pour charger une source, il faut indiquer :
- la source des données (un dossier par source dans le dossier `data`)
- en option, les dates sur lesquels charger la donnée

In [ ]:

loader.loadSource(Loader.COINCODEX, dateStart="2000-01-01", dateEnd="2024-01-01")

df = loader.get(Loader.COINCODEX, Loader.ADA)

df.tail()

,High,Low,Open,Close,Volume,Marketcap
Date,,,,,,
2022-08-13,0.592463,0.557968,0.559462,0.569894,1.162096e+09,1.955747e+10
2022-08-14,0.579933,0.544696,0.569312,0.550622,9.264294e+08,1.896335e+10
2022-08-15,0.568240,0.548788,0.549579,0.558032,8.294682e+08,1.888561e+10
2022-08-16,0.579608,0.532226,0.557456,0.535811,9.744968e+08,1.863630e+10
2022-08-17,0.542771,0.511639,0.537622,0.512444,7.755839e+08,1.809868e+10


In [ ]:
loader.loadSource(Loader.COINCODEX, forceEqualDate=True)
loader.loadSource(Loader.KAGGLE, forceEqualDate=True)

In [ ]:

dfAda = loader.data[Loader.COINCODEX][Loader.ADA]
dfBtc = loader.data[Loader.COINCODEX][Loader.BTC]

print("Ada from ", dfAda.index[0], " to ", dfAda.index[-1])
print("Btc from ", dfBtc.index[0], " to ", dfBtc.index[-1])

Ada from  2017-10-01  to  2022-08-17
Btc from  2017-10-01  to  2022-08-17


Pour charger toutes les données :

In [ ]:
loader.loadSources()